In [12]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate

# Define the prompt template
prompt = PromptTemplate.from_template("A {word} is a")

# Initialize the HuggingFacePipeline with the specified model and token limit
llm = HuggingFacePipeline.from_model_id(
    model_id="openai-community/gpt2",
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 150,
    },
)


# Define a function to generate the prompt and get the response
def generate_definition(word):
    prompt_text = prompt.format(word=word)
    response = llm(prompt_text)
    return response


# Test the function
response = generate_definition("potato")
print(response)

A potato is a small, white, oval shaped bird with brown wings that is often considered the officer's best bird during the hunting season. It is a versatile, highly adaptable bird, and is used on land in most countries with its natural attributes and in hunting and
